In [21]:
import pandas as pd
import numpy as np
import random
import string
import os
import csv
import kaggle
import contextlib
import io
import zipfile
import tensorflow as tf
import torch
import copy
import textract
import pandas as pd
import glob

from tensorflow.keras.layers import Input, Dense, Concatenate, Dropout, Bidirectional, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from kaggle.api.kaggle_api_extended import KaggleApi
from keras.utils import to_categorical
from transformers import BertTokenizer, BertModel
from catboost import CatBoostClassifier
from numpy.random import choice as ch
from faker import Faker
from itertools import chain
from tqdm import tqdm

In [22]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [23]:
OKATO_REGIONS = '''01	Алтайский край
03	Краснодарский край
04	Красноярский край
05	Приморский край
07	Ставропольский край
08	Хабаровский край
10	Амурская область
11	Архангельская область
12	Астраханская область
14	Белгородская область
15	Брянская область
17	Владимирская область
18	Волгоградская область
19	Вологодская область
20	Воронежская область
22	Нижегородская область
24	Ивановская область
25	Иркутская область
26	Республика Ингушетия
27	Калининградская область
28	Тверская область
29	Калужская область
30	Камчатский край
32	Кемеровская область - Кузбасс
33	Кировская область
34	Костромская область
35	Республика Крым
36	Самарская область
37	Курганская область
38	Курская область
40	Город Санкт-Петербург город федерального значения	
41	Ленинградская область
42	Липецкая область
44	Магаданская область
45	Город Москва столица Российской Федерации город федерального значения	
46	Московская область
47	Мурманская область
49	Новгородская область
50	Новосибирская область
52	Омская область
53	Оренбургская область
54	Орловская область
56	Пензенская область
57	Пермский край
58	Псковская область
60	Ростовская область
61	Рязанская область
63	Саратовская область
64	Сахалинская область
65	Свердловская область
66	Смоленская область
67	Город федерального значения Севастополь	
68	Тамбовская область
69	Томская область
70	Тульская область
71	Тюменская область
73	Ульяновская область
75	Челябинская область
76	Забайкальский край
77	Чукотский автономный округ
78	Ярославская область
79	Республика Адыгея (Адыгея)
80	Республика Башкортостан
81	Республика Бурятия
82	Республика Дагестан
83	Кабардино-Балкарская Республика
84	Республика Алтай
85	Республика Калмыкия
86	Республика Карелия
87	Республика Коми
88	Республика Марий Эл
89	Республика Мордовия
90	Республика Северная Осетия-Алания
91	Карачаево-Черкесская Республика
92	Республика Татарстан (Татарстан)
93	Республика Тыва
94	Удмуртская Республика
95	Республика Хакасия
96	Чеченская Республика
97	Чувашская Республика - Чувашия
98	Республика Саха (Якутия)
99	Еврейская автономная область'''.split('\n')

In [24]:
regions = {}
for i in OKATO_REGIONS:
    row = i.split('\t')
    regions[row[0]] = row[1]

In [25]:
fake = Faker('ru_RU')
data = []

In [26]:
def generate_passport():
    region = random.choice(list(regions.keys()))
    year = str(random.choice([i for i in range(24)]))
    if len(year) == 1:
        year = f'0{year}'
    series = ''.join([str(random.randint(0, 9)) for i in range (6)])
    return f'{region} {year} {series}'

In [28]:
generate_passport()

'73 00 638321'

In [27]:
def generate_snils():
    nums = [
        random.randint(1, 1) if x == 0
        else '-' if x == 3
        else '-' if x == 7
        else ' ' if x == 11
        else random.randint(0, 9)
        for x in range(0, 12)
    ]

    cont = (nums[10] * 1) + (nums[9] * 2) + (nums[8] * 3) + \
           (nums[6] * 4) + (nums[5] * 5) + (nums[4] * 6) + \
           (nums[2] * 7) + (nums[1] * 8) + (nums[0] * 9)

    if cont in (100, 101):
        cont = '00'

    elif cont > 101:
        cont = (cont % 101)
        if cont in (100, 101): cont = '00'
        elif cont < 10: cont = '0' + str(cont)

    elif cont < 10: cont = '0' + str(cont)

    nums.append(cont)
    return ''.join([str(x) for x in nums])

In [93]:
generate_snils()

'174-582-548 00'

In [29]:
def ctrl_summ(nums, type):
    ctrl_type = {
        'n2_12': [7, 2, 4, 10, 3, 5, 9, 4, 6, 8],
        'n1_12': [3, 7, 2, 4, 10, 3, 5, 9, 4, 6, 8],
        'n1_10': [2, 4, 10, 3, 5, 9, 4, 6, 8],
    }
    n = 0
    l = ctrl_type[type]
    for i in range(0, len(l)):
        n += nums[i] * l[i]
    return n % 11 % 10

In [30]:
def generate_inn():
    l = ch([10, 12], p=[0.1, 0.9])
    nums = [
        random.randint(9, 9) if x == 0
        else random.randint(6, 6) if x == 1
        else random.randint(0, 9)
        for x in range(0, 9 if l == 10 else 10)
    ]

    if l == 10:
        n1 = ctrl_summ(nums, 'n1_10')
        nums.append(n1)

    elif l == 12:
        n2 = ctrl_summ(nums, 'n2_12')
        nums.append(n2)
        n1 = ctrl_summ(nums, 'n1_12')
        nums.append(n1)

    return ''.join([str(x) for x in nums])

In [31]:
generate_inn()

'968903856632'

In [33]:
temp = dict.fromkeys(set([fake.job() for i in range(10000)]))
jobs_list_fake = list(temp)

jobs_df = pd.read_csv('OKZ_jobs_list.csv', delimiter=',', header=11)[:3101]
jobs_df = jobs_df.iloc[:, 2:-3]
jobs_df = jobs_df.drop_duplicates()
jobs_df = jobs_df.reset_index(drop=True)

job_list = list(set.union(set(jobs_df['Наименование должности '].values), set(jobs_list_fake)))
job_list = list(set([item for item in job_list if type(item) == str]))

job_list = list(set([item.rstrip(' ') for item in job_list 
            if '(' not in item and 
            ')' not in item and 
            'Российской Федерации' not in item and
            'Федерального Собрания' not in item and
            'федерального органа' not in item
            ]))


def generate_job():
    return random.choice(job_list)

In [40]:
generate_job()

'Техник по стандартизации'

In [41]:
def generate_salary():
    # апрель 2019
    # https://dzen.ru/a/Xdz7rxjAYBYMxMeg, Росстат
    ranges = [
        (10000, 11280),
        (11280, 13000),
        (13000, 17000),
        (17000, 24820),
        (24820, 33900),
        (33900, 50000),
        (50000, 75000),
        (75000, 100000),
        (100000, 250000),
        (250000, 1000000)
    ]
    weights = [
        0.029,
        0.035,
        0.073,
        0.17,
        0.187,
        0.219,
        0.154,
        0.062,
        0.063,
        0.008
    ]
    cdf = np.cumsum(weights)
    u = random.random()
    index = np.searchsorted(cdf, u)
    salary = random.uniform(*ranges[index])
    return int(salary)


In [44]:
generate_salary()

35596

In [45]:
def generate_phone_number():
    patterns = [
        '+7##########',
        '+7 ### ### ## ##',
        '+7 ### ### ##-##',
        '+7 ### ###-## ##',
        '+7 ### ###-##-##',
        '+7 (###) ### ## ##',
        '+7 (###) ### ##-##',
        '+7 (###) ###-## ##',
        '+7 (###) ###-##-##',
        '8##########',
        '8 ### ### ## ##',
        '8 ### ### ##-##',
        '8 ### ###-## ##',
        '8 ### ###-##-##',
        '8 (###) ### ## ##',
        '8 (###) ### ##-##',
        '8 (###) ###-## ##',
        '8 (###) ###-##-##',
    ]
    return fake.bothify(random.choice(patterns))

In [46]:
generate_phone_number()

'+7 612 448-19 32'

In [114]:
num_records = 1000 # 2000 для ВКР, пока тестирую 1000 для преддипломной

In [115]:
data = []
for _ in range(num_records):
        gender = random.choice(["М", "Ж", 'Мужчина', 'Женщина', 'Муж', 'Жен'])
        full_name = fake.name_male() if gender in ['М', 'Муж', 'Мужчина'] else fake.name_female()
        birth_date = fake.date_of_birth(minimum_age=18, maximum_age=70)
        passport = generate_passport()
        address = random.choice([fake.address(), fake.address()[:-8]])
        phone = generate_phone_number()
        income = generate_salary()
        inn = generate_inn()
        snils = generate_snils()
        military_status = random.choice(['Годен', 'Не годен', 'Годен с ограничениями', 'Отсрочка', 'Альтернативная служба']) if gender in ['М', 'Муж', 'Мужчина'] else '-'
        job_experience = generate_job()
        marital_status = random.choice(["Холост", "Женат"]) if gender in ['М', 'Муж', 'Мужчина'] else random.choice(['Не замужем', 'Замужем'])
        education = random.choice(["Среднее", "Средне-специальное", "Высшее", "Незаконченное высшее"])

        data.append({
        "Полное имя": full_name,
        "Пол": gender,
        "Дата рождения": birth_date,
        "Паспорт": passport,
        "Место и адрес жительства": address,
        "Номер телефона": phone,
        "Сведения о доходах": income,
        "ИНН": inn,
        "СНИЛС": snils,
        "Отношение к воинской обязанности": military_status,
        "Сведения о трудовом стаже, предыдущих местах работы": job_experience,
        "Семейное положение": marital_status,
        "Данные об образовании, квалификации": education,
})

In [47]:
def generate_random_string():
    text = [random.choices(population=['?', '#', '!', ' '], weights=[0.7, 0.15, 0.1, 0.05], k=1) for i in range(random.randint(1,70))]
    temp = fake.bothify(''.join([item[0] for item in text]))

    translation_table = str.maketrans({
        ch: random.choice(f'абвгдеёжзийклмнопрстуфхцчшщъыьэюя{"абвгдеёжзийклмнопрстуфхцчшщъыьэюя".upper()}')
        for ch in string.ascii_letters
    })

    return temp.translate(translation_table)

In [48]:
generate_random_string()

'пыАЧмфЩЙ7к2А66Шбк ЙшмУЧчч1 Т5КпуьЩп4мТЩЖ'

In [50]:
def generate_random_onlyletters_string():
    text = [random.choices(population=['?', ' '], weights=[0.9, 0.1], k=1) for i in range(random.randint(1,70))]
    temp = fake.bothify(''.join([item[0] for item in text]))

    
    translation_table = str.maketrans({
        ch: random.choice(f'абвгдеёжзийклмнопрстуфхцчшщъыьэюя{"абвгдеёжзийклмнопрстуфхцчшщъыьэюя".upper()}')
        for ch in string.ascii_letters
    })

    return temp.translate(translation_table)

In [63]:
generate_random_onlyletters_string()

'э ыхджЗш Ж аосоЖуЦьчй ЦцЪсЪЪ ыСоыНуЖЦсЭшуузйЭсырйьЭ шлй лЦхЧйышчу'

In [64]:
def generate_INN_SNILS_like():
    text = [random.choices(population=['#', '!', ' '], weights=[0.7, 0.2, 0.1], k=1) for i in range(random.randint(6,14))]
    temp = [item[0] for item in text]

    text = fake.bothify(''.join(temp))
    ridx = random.randint(0, len(text) - 1)
    new_text = text[:ridx] + '-' + text[ridx + 1:]

    return new_text

In [68]:
generate_INN_SNILS_like()

'4-664846 007'

In [69]:
def generate_passport_mimic():
    base_text = list(generate_passport())
    random.shuffle(base_text)
    return ''.join(base_text)

In [70]:
generate_passport_mimic()

'312212 13 17'

In [72]:
def generate_INN_mimic():
    base_text = list(generate_inn())
    base_text = f'{random.choice([str(i) for i in range(0, 8)])}{generate_inn()[1:]}'
    return ''.join(base_text)

In [73]:
generate_INN_mimic()

'366533822720'

In [74]:
content = ''
with open('10000-russian-words-cyrillic-only.txt', 'r', encoding='utf-8') as f:
    content = f.readlines()
russian_words_dictionary = [i.replace('\n', '') for i in content]

In [75]:
def generate_words_string():
    word_count = random.randint(1, 4)
    digit_check = random.randint(0, 1)
    text = ''
    if digit_check:
        text = random.choices(population=russian_words_dictionary, k=word_count)
        digit_count = random.randint(1, 4)
        for i in range(digit_count):
            digit = ''.join(random.choices(population=[str(i) for i in range (10)], k=random.randint(1, 8)))
            text.append(digit)
        random.shuffle(text)
        return ' '.join(text)
    else:
        text = random.choices(population=russian_words_dictionary, k=word_count)
        return ' '.join(text)

In [104]:
generate_words_string()

'мужество пробуждение родина фиксировать'

In [105]:
mappings = {
    'Полное имя': ['full_name', 'name', 'FIO'],
    'Пол': ['gender', 'sex', 'm_or_f'],
    'Дата рождения': ['birth_date', 'date_of_birth', 'dob', 'birthdate', 'bdate'],
    'Паспорт': ['passport_number', 'passport_id', 'passport_series_and_number', 'passport', 'pass', 'pass_id', 'pass_num', 'pass_number'],
    'Место и адрес жительства': ['address', 'residence_address', 'living_address', 'living_place'],
    'Номер телефона': ['phone_number', 'phone', 'mobile_phone', 'pnumber', 'p_number', 'phone_n', 'number'],
    'Сведения о доходах': ['income_info', 'salary_info', 'earnings_info', 'income', 'salary', 'earnings'],
    'ИНН': ['inn', 'EIN', 'ein', 'INN', 'tin', 'TIN'],
    'СНИЛС': ['snils', 'SNILS', 'inila', 'INILA', 'IIAN', 'iian'],
    'Отношение к воинской обязанности': ['military_service_info', 'military_status', 'mil_status', 'military_s', 'military_service_status'],
    'Сведения о трудовом стаже, предыдущих местах работы': ['work_experience', 'previous_jobs', 'employment_history', 'job_experience', 'work_years', 'job_years', 'employment_years'],
    'Семейное положение': ['marital_status', 'family_status', 'marital', 'marriage', 'fam_status'],
    'Данные об образовании, квалификации': ['education_info', 'qualification_info', 'degree_info', 'education', 'qualification', 'degree']
}
mappings_label = {
    'Полное имя': 1,
    'Пол': 2,
    'Дата рождения': 3,
    'Паспорт': 4,
    'Место и адрес жительства': 5,
    'Номер телефона': 6,
    'Сведения о доходах': 7,
    'ИНН': 8,
    'СНИЛС': 9,
    'Отношение к воинской обязанности': 10,
    'Сведения о трудовом стаже, предыдущих местах работы': 11,
    'Семейное положение': 12,
    'Данные об образовании, квалификации': 13,
    'Не является ПДн': 14
}
mappings_types = {
    'Полное имя': ['char', 'varchar', 'text'],
    'Пол': ['char', 'varchar', 'text'],
    'Дата рождения': ['char', 'varchar', 'text', 'date', 'time', 'datetime', 'timestamp'],
    'Паспорт': ['char', 'varchar', 'text'],
    'Место и адрес жительства': ['char', 'varchar', 'text'],
    'Номер телефона': ['char', 'varchar', 'text'], # числовые? но только те, что без ' ', '-', '()'
    'Сведения о доходах': ['char', 'varchar', 'text', 'integer', 'numeric', 'decimal'], # float? double?
    'ИНН': ['char', 'varchar', 'text', 'integer', 'numeric', 'decimal'],
    'СНИЛС': ['char', 'varchar', 'text'],
    'Отношение к воинской обязанности': ['char', 'varchar', 'text'],
    'Сведения о трудовом стаже, предыдущих местах работы': ['char', 'varchar', 'text'],
    'Семейное положение': ['char', 'varchar', 'text'],
    'Данные об образовании, квалификации': ['char', 'varchar', 'text'],
    'Не является ПДн': ['char', 'varchar', 'text']
}

In [116]:
df = pd.DataFrame(data)

In [118]:
df.head(5)

,Полное имя,Пол,Дата рождения,Паспорт,Место и адрес жительства,Номер телефона,Сведения о доходах,ИНН,СНИЛС,Отношение к воинской обязанности,"Сведения о трудовом стаже, предыдущих местах работы",Семейное положение,"Данные об образовании, квалификации"
0,Леон Иосифович Логинов,Муж,1981-04-15,40 01 465203,"с. Чебоксары, ул. Коминтерна, д. 67, 400537",8 985 299-88 72,28245,9604448627,141-110-413 76,Альтернативная служба,Зенитчик,Женат,Среднее
1,Святослав Архипович Ситников,Муж,1989-09-28,93 21 801819,"ст. Егорьевск, бул. Красноармейский, д. 4",+7 550 615-78-45,17719,962920637407,131-817-129 36,Не годен,Санитар,Женат,Средне-специальное
2,Юлиан Тарасович Цветков,Муж,1984-05-20,18 05 935386,"клх Йошкар-Ола, пр. Заречный, д. 4 к. 4",+7 (592) 351 78 72,11216,965379546665,190-025-089 35,Не годен,Директор киностудии,Женат,Среднее
3,Матвей Игоревич Селиверстов,Муж,1994-07-11,24 17 861832,"ст. Льгов, бул. Прудовой, д. 37, 052678",+7 076 462 12-92,49982,967708211886,103-001-715 62,Годен с ограничениями,Инженер по радиосвязи,Женат,Среднее
4,Олимпиада Георгиевна Антонова,Женщина,1986-06-08,61 22 076149,"г. Краснодар, бул. Индустриальный, д. 1/3 стр. 5",+7 964 680 20 25,12000,963814194522,107-987-124 90,-,Ревизор коммерческий,Замужем,Средне-специальное


In [119]:
df['Сведения о трудовом стаже, предыдущих местах работы'].str.len().max()

121

In [121]:
df.shape

(1000, 13)

In [120]:
df_rdy = pd.DataFrame({
    # 'name': [], 
    'type': [], 
    'data': [], 
    'label': []
})

for col in df.columns: 
    for i in range(num_records): 
        name = random.choice(mappings[col]) 
        label = mappings_label[list(filter(lambda x: name in mappings[x], mappings))[0]]

        df_rdy = df_rdy.append({
        # 'name': random.choice(mappings[col]), 
        'type': random.choice(mappings_types[col]), # str, integer, bigint, datetime, numeric
        'data': df[col].iloc[i],
        'label': label
        }, ignore_index=True
        )

In [122]:
df_rdy

,type,data,label
0,char,Леон Иосифович Логинов,1.0
1,text,Святослав Архипович Ситников,1.0
2,varchar,Юлиан Тарасович Цветков,1.0
3,varchar,Матвей Игоревич Селиверстов,1.0
4,varchar,Олимпиада Георгиевна Антонова,1.0
...,...,...,...
12995,text,Средне-специальное,13.0
12996,char,Незаконченное высшее,13.0
12997,varchar,Среднее,13.0
12998,char,Незаконченное высшее,13.0


In [213]:
import tika
from tika import parser

defpath = 'books\\'
books = [
    "voina-i-mir.epub",
    "prestuplenie-i-nakazanie.epub",
    "Mikhail_Bulgakov__Master_I_Margarita.epub",
    "Oruell-1984.epub"
]
#

for fileIn in books:
    parsed = parser.from_file(defpath + fileIn)
    content = parsed["content"]
    fileOut = fileIn.replace('epub', 'txt')

    with open(defpath +fileOut, 'w', encoding='utf-8') as fout:
        fout.write(content)

In [256]:
import nltk
import random

nltk.download('punkt')
tokenizer_t = nltk.sent_tokenize

targets = [book.replace('epub', 'txt') for book in books]
sentences = []
for target in targets:
    temp = tokenizer_t(content)
    for i in temp:
        sentences.append(i)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\minichazer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [257]:
sentences = [
    sentence.replace('\n', '').replace('\xa0—', '').replace('\xa0', '')
    for sentence in sentences
    ]

In [277]:
def get_random_sentence(sentence_length: int):
    while True:
        temp = random.choice(sentences)
        if len(temp) < sentence_length:
            return temp


In [294]:
get_random_sentence(128)

'Распоряжения явиться на службу, которые уже неслись из телекранов, были излишни.'

In [295]:
generation_methods = {
    'just_string': generate_random_onlyletters_string,
    'string_with_digits': generate_random_string,
    'INN-like': generate_INN_SNILS_like,
    'random_words': generate_words_string,
    'mimic_INN': generate_INN_mimic,
    'mimic_passport': generate_passport_mimic,
    'get_random_sentence': lambda: get_random_sentence(128)
}

In [297]:
for col in df.columns:
    for i in range(num_records * 2):
        method_choice = random.choice(list(generation_methods.keys()))
        text = generation_methods[method_choice]()
        
        df_rdy = df_rdy.append({
            'type': random.choice(mappings_types['Не является ПДн']),  # str, integer, bigint, datetime, numeric
            'data': text,
            'label': 14
        }, ignore_index=True)

In [298]:
df_rdy

,type,data,label
0,char,Леон Иосифович Логинов,1.0
1,text,Святослав Архипович Ситников,1.0
2,varchar,Юлиан Тарасович Цветков,1.0
3,varchar,Матвей Игоревич Селиверстов,1.0
4,varchar,Олимпиада Георгиевна Антонова,1.0
...,...,...,...
38995,text,730 -1,14.0
38996,char,Разве в августе не темнеет к двадцати одному ч...,14.0
38997,text,9546 948087,14.0
38998,text,276 -,14.0


In [300]:
tokenizer = BertTokenizer.from_pretrained(
    'google-bert/bert-base-uncased', 
    return_attention_mask=False,
    return_token_type_ids=False
)
model_bert = BertModel.from_pretrained('google-bert/bert-base-uncased')

In [301]:
print(df_rdy.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39000 entries, 0 to 38999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   type    39000 non-null  object 
 1   data    39000 non-null  object 
 2   label   39000 non-null  float64
dtypes: float64(1), object(2)
memory usage: 914.2+ KB
None


In [308]:
df_rdy['type'] = df_rdy['type'].astype(str)
df_rdy['data'] = df_rdy['data'].astype(str)
df_rdy_part = df_rdy.sample(n=num_records * 39) # 26
df_rdy_part.reset_index(drop=True, inplace=True)
df_rdy_part

,type,data,label
0,text,41 14 797174,4.0
1,char,9118 0 27187,14.0
2,char,ДЭАВИж29АиюююжшХГ2ВшЕкИм2з,14.0
3,varchar,6 -3477,14.0
4,varchar,"Он отпустил плечи Уинстона, слегка толкнув его...",14.0
...,...,...,...
38995,decimal,967502198039,8.0
38996,varchar,7ьяХх3СЗя 0Зи57оЖ6Ии0ЩоЁ,14.0
38997,text,-,10.0
38998,time,2002-07-12,3.0


In [309]:
lb = LabelEncoder()
df_rdy_part['type'] = lb.fit_transform(df_rdy_part['type'])
df_rdy_part

,type,data,label
0,6,41 14 797174,4.0
1,0,9118 0 27187,14.0
2,0,ДЭАВИж29АиюююжшХГ2ВшЕкИм2з,14.0
3,9,6 -3477,14.0
4,9,"Он отпустил плечи Уинстона, слегка толкнув его...",14.0
...,...,...,...
38995,3,967502198039,8.0
38996,9,7ьяХх3СЗя 0Зи57оЖ6Ии0ЩоЁ,14.0
38997,6,-,10.0
38998,7,2002-07-12,3.0


In [304]:
def one_hot_encode(text, num_classes=255):
    one_hot = to_categorical(text, num_classes=num_classes)
    return one_hot

In [310]:
df_rdy_part['type'] = df_rdy_part['type'].apply(one_hot_encode)
df_rdy_part

,type,data,label
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...",41 14 797174,4.0
1,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",9118 0 27187,14.0
2,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",ДЭАВИж29АиюююжшХГ2ВшЕкИм2з,14.0
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",6 -3477,14.0
4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","Он отпустил плечи Уинстона, слегка толкнув его...",14.0
...,...,...,...
38995,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",967502198039,8.0
38996,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",7ьяХх3СЗя 0Зи57оЖ6Ии0ЩоЁ,14.0
38997,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...",-,10.0
38998,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...",2002-07-12,3.0


In [351]:
sample_size = int(num_records * 39)
df_rdy_part_small = df_rdy_part.sample(sample_size)
df_rdy_part_small.reset_index(drop=True, inplace=True)
df_rdy_part_small

,type,data,label
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","Но это до войны, конечно.",14.0
1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Тренер,11.0
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","с. Серафимович, наб. Путейская, д. 318 к. 9/1",5.0
3,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Среднее,13.0
4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","ст. Усть-Ордынский, ш. Овражное, д. 9/6 стр. 251",5.0
...,...,...,...
38995,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",381027 077 9,14.0
38996,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",-,10.0
38997,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0 -9,14.0
38998,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","Наконец он вполне убедился, что полнеет: бедра...",14.0


In [352]:
def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

In [353]:
new_df = pd.DataFrame(columns=df_rdy_part_small.columns)
block_size = int(df_rdy_part_small.shape[0] / 10) - 1
blocks = [df_rdy_part_small[i:i+block_size] for i in range(0, len(df_rdy_part_small), block_size)]

In [354]:
blocks[10]

,type,data,label
38990,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...",062530433951,14.0
38991,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1815-0,14.0
38992,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...",Адресных книг нет.,14.0
38993,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...",01 10 895569,4.0
38994,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Хф23йноэ1Аннбо1Вбс 4нъйЪВю ВлобчШДиоХЦш,14.0
38995,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",381027 077 9,14.0
38996,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",-,10.0
38997,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0 -9,14.0
38998,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","Наконец он вполне убедился, что полнеет: бедра...",14.0
38999,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",28 06 690033,4.0


In [355]:
def process_block(block: pd.DataFrame, model, tokenizer):
    texts = block['data'].tolist()
    tokenized_texts = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in tokenized_texts.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    embeddings = embeddings.cpu().numpy()
    temp = block.copy()
    temp['data'] = embeddings.tolist()
    return pd.DataFrame(temp)

In [356]:
process_block(blocks[10], model_bert, tokenizer)

,type,data,label
38990,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[-0.059342049062252045, 0.016148101538419724, ...",14.0
38991,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.05252661183476448, -0.009448813274502754, ...",14.0
38992,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[-0.0353299155831337, 0.021005986258387566, -0...",14.0
38993,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[-0.03138775750994682, 0.017057059332728386, -...",4.0
38994,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.024611908942461014, -4.856078157899901e-05...",14.0
38995,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.03346395492553711, 0.02389773726463318, -0...",14.0
38996,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.01887720637023449, -0.007937731221318245, ...",10.0
38997,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.04495000094175339, -0.0019072296563535929,...",14.0
38998,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[-0.03291013464331627, 0.01871037669479847, -0...",14.0
38999,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.03728269040584564, 0.015542114153504372, -...",4.0


In [357]:
process_block(blocks[10], model_bert, tokenizer).head(1).iloc[0]

type     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...
data     [-0.059342049062252045, 0.016148101538419724, ...
label                                                 14.0
Name: 38990, dtype: object

In [358]:
for i, block in enumerate(tqdm(blocks)):
    new_block = process_block(block, model_bert, tokenizer)
    new_block.to_pickle(f'df-chunks\\{i}_chunk.pkl')

100%|██████████| 11/11 [1:01:36<00:00, 336.02s/it]


In [360]:
dfs = []
for i in range(11):
    filename = f'df-chunks\\{i}_chunk.pkl'
    df = pd.read_pickle(filename)
    dfs.append(df)
df_rdy_part_small = pd.concat(dfs, ignore_index=True)
df_rdy_part_small

,type,data,label
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.026118218898773193, 0.018988119438290596, ...",14.0
1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.03868452087044716, -0.0006962965126149356,...",11.0
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[-0.03977047652006149, 0.037099793553352356, -...",5.0
3,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.029298972338438034, 0.005822142586112022, ...",13.0
4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.054089609533548355, 0.016389301046729088, ...",5.0
...,...,...,...
38995,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.03346395492553711, 0.02389773726463318, -0...",14.0
38996,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.01887720637023449, -0.007937731221318245, ...",10.0
38997,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.04495000094175339, -0.0019072296563535929,...",14.0
38998,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[-0.03291013464331627, 0.01871037669479847, -0...",14.0


In [361]:
save_point = df_rdy_part_small.copy()

In [362]:
df_rdy_part_small['label']

0        14.0
1        11.0
2         5.0
3        13.0
4         5.0
         ... 
38995    14.0
38996    10.0
38997    14.0
38998    14.0
38999     4.0
Name: label, Length: 39000, dtype: float64

In [363]:
def one_hot_encode(text, num_classes=15):
    one_hot = to_categorical(text, num_classes=num_classes)
    return one_hot

In [364]:
def remove_first_element(arr):
    return arr[1:]

In [365]:
df_rdy_part_small['label'] = df_rdy_part_small['label'].astype({'label':int})
df_rdy_part_small['label'] = df_rdy_part_small['label'].apply(one_hot_encode)
df_rdy_part_small['label'] = df_rdy_part_small['label'].apply(remove_first_element)

In [366]:
df_rdy_part_small

,type,data,label
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.026118218898773193, 0.018988119438290596, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.03868452087044716, -0.0006962965126149356,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[-0.03977047652006149, 0.037099793553352356, -...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
3,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.029298972338438034, 0.005822142586112022, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.054089609533548355, 0.016389301046729088, ...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...
38995,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.03346395492553711, 0.02389773726463318, -0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
38996,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.01887720637023449, -0.007937731221318245, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
38997,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.04495000094175339, -0.0019072296563535929,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
38998,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[-0.03291013464331627, 0.01871037669479847, -0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [367]:
df_rdy_part_small['label'].shape

(39000,)

In [368]:
i_type = Input(shape=(255,), dtype='float32', name='i_type') # type
i_data = Input(shape=(768,), dtype='float32', name='i_data') # data
type_dense = Dense(128, activation='relu')(i_type)
data_dense = Dense(128, activation='relu')(i_data)
concat = Concatenate()([type_dense, data_dense])
dropout = Dropout(0.2)(concat)
output = Dense(14, activation='softmax')(dropout)
model = Model(inputs=[i_type, i_data], outputs=output)

In [369]:
model.summary()

Model: "model_46"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 i_type (InputLayer)         [(None, 255)]                0         []                            
                                                                                                  
 i_data (InputLayer)         [(None, 768)]                0         []                            
                                                                                                  
 dense_138 (Dense)           (None, 128)                  32768     ['i_type[0][0]']              
                                                                                                  
 dense_139 (Dense)           (None, 128)                  98432     ['i_data[0][0]']              
                                                                                           

In [370]:
model.compile(
    optimizer='adam', 
    loss='categorical_crossentropy', 
    metrics=[
        tf.keras.metrics.Precision(), 
        tf.keras.metrics.Recall(), 
        'accuracy'])

In [371]:
train_df, test_df = train_test_split(df_rdy_part_small, test_size=0.2, random_state=42)

X_train = train_df[['type', 'data']]
y_train = train_df['label']

X_test = test_df[['type', 'data']]
y_test = test_df['label']

type_train = np.stack(X_train['type'].values)
data_train = np.stack(X_train['data'].values)

y_train = np.stack(y_train.values)

type_test = np.stack(X_test['type'].values)
data_test = np.stack(X_test['data'].values)

y_test = np.stack(y_test.values)

In [372]:
data_train.shape

(31200, 768)

In [373]:
history = model.fit(x=[type_train, data_train], y=y_train,
                    epochs=10,
                    validation_split=0.2)

Epoch 1/10
780/780 [==============================] - 4s 5ms/step - loss: 0.7710 - precision_46: 0.8551 - recall_46: 0.6879 - accuracy: 0.7793 - val_loss: 0.3556 - val_precision_46: 0.9353 - val_recall_46: 0.8197 - val_accuracy: 0.9139
Epoch 2/10
780/780 [==============================] - 3s 4ms/step - loss: 0.2706 - precision_46: 0.9467 - recall_46: 0.8846 - accuracy: 0.9221 - val_loss: 0.1849 - val_precision_46: 0.9522 - val_recall_46: 0.9220 - val_accuracy: 0.9327
Epoch 3/10
780/780 [==============================] - 3s 4ms/step - loss: 0.1633 - precision_46: 0.9616 - recall_46: 0.9357 - accuracy: 0.9504 - val_loss: 0.1266 - val_precision_46: 0.9596 - val_recall_46: 0.9484 - val_accuracy: 0.9530
Epoch 4/10
780/780 [==============================] - 3s 4ms/step - loss: 0.1182 - precision_46: 0.9719 - recall_46: 0.9560 - accuracy: 0.9641 - val_loss: 0.0933 - val_precision_46: 0.9770 - val_recall_46: 0.9612 - val_accuracy: 0.9713
Epoch 5/10
780/780 [==============================] - 3s

In [374]:
model.evaluate(x=[type_test, data_test], y=y_test)

244/244 [==============================] - 1s 2ms/step - loss: 0.0528 - precision_46: 0.9833 - recall_46: 0.9823 - accuracy: 0.9824


[0.05281560495495796,
 0.9833162426948547,
 0.9823076725006104,
 0.9824358820915222]

In [87]:
# model.save('model_20795.h5')

c:\Users\minichazer\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# from keras.models import load_model

# model = load_model('model.h5')

In [ ]:
# precision = history.history['precision_1']
# recall = history.history['recall_1']
# val_precision = history.history['val_precision_1']
# val_recall = history.history['val_recall_1']

# train_f1 = 2 * (precision * recall) / (precision + recall)
# val_f1 = 2 * (val_precision * val_recall) / (val_precision + val_recall)

# print('Training F1-score:', np.mean(train_f1))
# print('Validation F1-score:', np.mean(val_f1))

In [612]:
# y_pred = model.predict(x=[type_test, data_test], batch_size=64, verbose=1)
# y_pred_bool = np.argmax(y_pred, axis=1)
# # y_pred_bool[2]

# print(classification_report(y_test, y_pred_bool))

In [613]:
# data_test

In [375]:
eval_test_df = pd.DataFrame(columns=['type', 'data'])

vals = {
    'Даничкин Антон Сергеевич': 'char',
    'Безопасность Безопасность Безопасность': 'text',

    'Муж': 'char',
    'Х': 'text',

    '01-01-1990': 'char',
    '12 56 78': 'text',

    '72 20 567123': 'char',
    '73 40593 848583468': 'text',

    'ЯНАО, г. Саеехард, ул. Кошшшшкино, д. 500': 'varchar',
    'Где-то там, 620, может быть и не там, да и не совсем уверен': 'text',

    '+7 922 28 60 923': 'char',
    '2 147 483 647': 'text',

    '70000': 'char',
    '250000000000000': 'text',

    '963538512189': 'char',
    '123123456456': 'text',

    '146-533-459 14': 'char',
    '123-123-123-123': 'text',

    'Годен с ограничениями': 'char',
    'Это никуда не годится': 'text',

    'Блогер': 'char',
    'Размышления и полет фантазии': 'text',

    'Не замужем': 'char',
    'Под окном': 'text',

    'Незаконченное высшее': 'char',
    'Два маленьких, два больших и два средних': 'text',

    
    # 'Инфоркмационная безорпасность': 'text',

    # '12': 'numeric',
    # '188-666-123 29': 'text',
    # 'г. Тюмень, ул. Приколов, д. 123': 'char',
    # '+79222860987': 'text',
    # 'Преподаватель истории': 'char',
    # 'Блогер': 'varchar',
    # '0.125832952983745': 'text',
    # '245000': 'integer',
    # f'3{generate_inn()[1:]}': 'text',
    # f'{generate_inn()}': 'text',
    # '1488-1377-3228-1234': 'text',
    
}

'''960538612109	142-531-429 28	Годен с ограничениями	Инженер по учету сырья и готовой продукции	Женат	Незаконченное высшее'''

for key, value in vals.items():
    eval_test_df = eval_test_df.append({
        'data': key,
        'type': value,
    }, ignore_index=True)

In [376]:
eval_test_df = pd.DataFrame(columns=['type', 'data'])

vals = {
    'Даничкин Антон Сергеевич': 'char',
    'Безопасность Безопасность Безопасность': 'text',
    'Муж': 'char',
    'Х': 'text',
    '01-01-1990': 'char',
    '12 56 78': 'text',
    '72 20 567123': 'char',
    '73 40593 848583468': 'text',
    'ЯНАО, г. Саеехард, ул. Кошшшшкино, д. 500': 'varchar',
    'Где-то там, 620, может быть и не там, да и не совсем уверен': 'text',
    '+7 922 28 60 923': 'char',
    '2 147 483 647': 'text',
    '70000': 'char',
    '250000000000000': 'text',
    '963538512189': 'char',
    '123123456456': 'text',
    '146-533-459 14': 'char',
    '123-123-123-123': 'text',
    'Годен с ограничениями': 'char',
    'Это никуда не годится': 'text',
    'Блогер': 'char',
    'Размышления и полет фантазии': 'text',
    'Не замужем': 'char',
    'Под окном': 'text',
    'Незаконченное высшее': 'char',
    'Два маленьких, два больших и два средних': 'text',
}

for key, value in vals.items():
    eval_test_df = eval_test_df.append({
        'data': key,
        'type': value,
    }, ignore_index=True)

In [377]:
eval_test_df

,type,data
0,char,Даничкин Антон Сергеевич
1,text,Безопасность Безопасность Безопасность
2,char,Муж
3,text,Х
4,char,01-01-1990
5,text,12 56 78
6,char,72 20 567123
7,text,73 40593 848583468
8,varchar,"ЯНАО, г. Саеехард, ул. Кошшшшкино, д. 500"
9,text,"Где-то там, 620, может быть и не там, да и не ..."


In [378]:
eval_test_df_original = eval_test_df.copy()

In [379]:
eval_test_df['type'] = lb.fit_transform(eval_test_df['type'])
eval_test_df['type'] = eval_test_df['type'].apply(one_hot_encode)
eval_test_df['data'] = eval_test_df['data'].apply(lambda x: embed_bert_cls(x, model_bert, tokenizer))

type_eval = np.stack(eval_test_df['type'].values)
data_eval = np.stack(eval_test_df['data'].values)

type_eval = np.zeros((type_eval.shape[0], 255), dtype=np.float32)
type_eval[:, :type_eval.shape[1]] = type_eval

In [380]:
predictions = model.predict(
    {
        'i_type': type_eval, 
        'i_data': data_eval
    })

# predictions_cb = model_cb.predict(
#     {
#         'i_type': type_eval, 
#         'i_data': data_eval
#     })

1/1 [==============================] - 0s 135ms/step


In [381]:
predictions_list = predictions.tolist()

In [393]:
predictions_df = pd.DataFrame(columns=['type', 'data', 'prediction'])

for i in range(len(predictions_list)):
    col1, col2 = eval_test_df_original['type'][i], eval_test_df_original['data'][i]
    sorted_indices = sorted(range(len(predictions_list[i])), key=lambda k: predictions_list[i][k], reverse=True)
    filtered_indices = [idx for idx in sorted_indices if predictions_list[i][idx] >= 0.3]
    predictions_str = ', '.join([f'{list(mappings_label.keys())[list(mappings_label.values()).index(idx+1)]} - {predictions_list[i][idx]:.4f}' for idx in filtered_indices])
    predictions_df = predictions_df.append({
        'type': col1,
        'data': col2,
        'prediction': predictions_str
    }, ignore_index=True)

predictions_df

,type,data,prediction
0,char,Даничкин Антон Сергеевич,Полное имя - 0.9959
1,text,Безопасность Безопасность Безопасность,Не является ПДн - 0.9998
2,char,Муж,Пол - 0.9967
3,text,Х,Не является ПДн - 0.9992
4,char,01-01-1990,Дата рождения - 0.9666
5,text,12 56 78,Не является ПДн - 0.8857
6,char,72 20 567123,Паспорт - 0.9481
7,text,73 40593 848583468,Не является ПДн - 0.9740
8,varchar,"ЯНАО, г. Саеехард, ул. Кошшшшкино, д. 500",Место и адрес жительства - 0.9997
9,text,"Где-то там, 620, может быть и не там, да и не ...",Не является ПДн - 0.9998


In [32]:
%load_ext tensorboard

In [16]:
import datetime
from tensorboard.plugins.hparams import api as hp

In [394]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([64, 128, 256]))
HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0.2, 0.3, 0.4]))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['rmsprop', 'nadam', 'adam', 'adamax']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

In [395]:
def train_test_model(hparams):
    i_type = Input(shape=(255,), dtype='float32', name='i_type') # type
    i_data = Input(shape=(768,), dtype='float32', name='i_data') # data
    type_dense = Dense(hparams[HP_NUM_UNITS], activation='relu')(i_type)
    data_dense = Dense(hparams[HP_NUM_UNITS], activation='relu')(i_data)
    concat = Concatenate()([type_dense, data_dense])
    dropout = Dropout(hparams[HP_DROPOUT])(concat)
    output = Dense(14, activation='softmax')(dropout)
    model = Model(inputs=[i_type, i_data], outputs=output)

    model.compile(
        optimizer=hparams[HP_OPTIMIZER], 
        loss='categorical_crossentropy', 
        metrics=[
            tf.keras.metrics.Precision(), 
            tf.keras.metrics.Recall(), 
            'accuracy'])

    model.fit(x=[type_train, data_train], y=y_train, epochs=4)
    accuracy = model.evaluate([type_test, data_test], y_test)[-1]
    return accuracy
        

In [19]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [396]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
  for dropout_rate in HP_DROPOUT.domain.values:
    for optimizer in HP_OPTIMIZER.domain.values:
      hparams = {
          HP_NUM_UNITS: num_units,
          HP_DROPOUT: dropout_rate,
          HP_OPTIMIZER: optimizer,
      }
      run_name = "run-%d" % session_num
      # run_name = "run-%d" % session_num
      print('--- Starting trial: %s' % run_name)
      print({h.name: hparams[h] for h in hparams})
      run('logs/hparam_tuning/' + run_name, hparams)
      session_num += 1

--- Starting trial: run-0
{'num_units': 64, 'dropout': 0.2, 'optimizer': 'adam'}
Epoch 1/4
975/975 [==============================] - 3s 2ms/step - loss: 0.8585 - precision_47: 0.8427 - recall_47: 0.6612 - accuracy: 0.7527
Epoch 2/4
975/975 [==============================] - 2s 2ms/step - loss: 0.3377 - precision_47: 0.9410 - recall_47: 0.8425 - accuracy: 0.9039
Epoch 3/4
975/975 [==============================] - 2s 2ms/step - loss: 0.2085 - precision_47: 0.9547 - recall_47: 0.9157 - accuracy: 0.9369
Epoch 4/4
244/244 [==============================] - 0s 1ms/step - loss: 0.1099 - precision_47: 0.9732 - recall_47: 0.9581 - accuracy: 0.9651
--- Starting trial: run-1
{'num_units': 64, 'dropout': 0.2, 'optimizer': 'adamax'}
Epoch 1/4
975/975 [==============================] - 2s 2ms/step - loss: 1.1331 - precision_48: 0.7656 - recall_48: 0.6079 - accuracy: 0.6863
Epoch 2/4
975/975 [==============================] - 2s 2ms/step - loss: 0.6231 - precision_48: 0.8904 - recall_48: 0.7028 - a

In [ ]:
%tensorboard --logdir logs/hparam_tuning

# Результаты относительно разных гиперпараметров 

<div>
<img src="images\results_list.png" width="800"/>
</div>

# График результатов
<div>
<img src="images\results_table.png" width="800"/>
</div>